# Transform invalid XML files into valid XML


In [1]:
import re
import os
from lxml import etree

## Merge TXT files into one file per volume


In [ ]:
# recoller les TRx-xxx.txt en TRx.txt

inputpath = os.path.join('..','data','1743_LeRobert', 'input')
outputpath = os.path.join('..', 'data', '1743_LeRobert', 'txt')

for i in range(1, 7):
    # get files list in inputpath starting with TRi-xxx.txt
    files = sorted(os.listdir(inputpath))
    content = ""
    for filename in files:
        if filename.startswith(f'TR{i}-'):
            with open(os.path.join(inputpath, filename), 'r', encoding='latin-1') as f:
                content += f.read()
  
    with open(os.path.join(inputpath, 'TR'+str(i)+'.txt'), 'w', encoding='utf-8') as f:
        f.write(content)


## Convert TXT files (containing invalid XML structure) into XML files

In [2]:
# Question des petites majuscules
def get_1st_page_T5():
    content = "<article ID=250376787>\n<Nat_Art>Trévoux 1743</Nat_Art>\n<G><vedette>PÉ pour Pierre,</vedette></G> Il y a en France une Abbaye nommée <I>S. Pé de Generès</I>, en Latin <I>S. Petri de Generofo</I>. Pierre Fournier, Procureur au Parlement, signoit P. Fournier, pour se distinguer de quelques-uns de ses confrères qui portoient aussi le nom de Fournier : c'est pourquoi on l'appeloit ordinairement Pé-Fournier."
    content += "Tous les Procureurs qui ont des confrères de même nom qu'eux, se distinguent ainsi par la première lettre de leur nom de batême. Dans la comédie Italienne d'Arlequin Procureur, Arlequin, pour imiter ce vers,"
    content += "<I>Et dont les Cicerons se font chez Pé-Fournier.</I>"
    content += "se nommoit Pé-Arlequin. <I>Brossette sur le vers 124. de la I. sat. de Boileau.</I>\n</article>"

    content += "\n<article ID=250376788>\n<Nat_Art>Trévoux 1743</Nat_Art>\n<G><vedette>PEAAGKIRKE,</vedette></G> Prononcez <I>Péguire</I>, &amp; Voyez Pégue. \n</article>\n"
    
    content += "<article ID=250376789>\n<Nat_Art>Trévoux 1743</Nat_Art>\n<G><vedette>PÉAGE,</vedette></G> <cat_gra>f. m.</cat_gra> Il s'est dit autrefois en général de toutes sortes d'impôts qui se payoient sur les machandises qu'on transportoit d'un lieu à un autre."
    content += "<I>Vectigal, tributum, portorium.</I> Maintenant il se dit d'un droit qu'on prend sur les voitures des marchandises pour l'entretien des grands chemins."
    content += "Diverses Coutumes l'accordent aux Châtelains ; &amp; elles appellent <I>cehmins péageux</I>, les chemins dont la réparation doit être faite par les Châtelains, ou autres ayant droits de <I>péage</I>."
    content += "La plûpart des Seigneurs s'attribuent des droits de <I>péage</I> sur leurs terres, sous prétexte d'entretenir les chemins, les ponts &amp; chaussées."
    content += "Anciennement ceux qui tenoient ce droit, devoient rendre les chemins sûrs, &amp; répondre des vols faits aux passans entre deux soleils. Cela s'observe encore en quelques endroits d'Angleterre &amp; d'Italie, où il y a des gardes qu'on appelle <I>Stationnaires</I>, établis pour la sûreté des marchands, &amp; entre autres à Terracine sur le chemin de Rome à Naples."
    content += "Il y a une Ordonnance de 1570 portant abolition de tous <I>péages</I> établis depuis 100 ans sur la rivière de Loire au profit du Roi ; &amp; injonction à tous autres prétendants droit de <I>péage</I>, de produire leurs titres au Parlement."
    content += "La plûpart des <I>péage</I> sont de pures usurpations. L'Ordonnance de 552 enjoint aux Seigneurs qui ont droit de <I>péage</I> d'entretenir les ponts &amp; passages."
    content += "Le <I>péage</I> est appelé de divers noms dans les Coutumes &amp; les Ordonnances. On le nomme <I>barrage</I> aux entrées des bourgs &amp; des villes ; <I>pontenage</I> au passage des ponts, <I>billette</I>, ou <I>branchière</I> aux passage de campagne, où l'on a mus pour signal un petit billot de bois, attaché à une branche."
    content += "On l'appelle quelquefois <I>coutume</I>, ou <I>droit établi sans titre</I> ; quelquefois <I>prévôté</I>, ou <I>menu droit casuel</I> ; &amp; quelquefois <I>travers</I>, qui est un droit qui ne se paye que sur la frontière."

    content += "\nLes Enfans de France &amp; Princes du sang royal, pour leurs provisions, font exempts de <I>péage</I> par tout le Royaume par privilège."
    content += "Il s'en trouve un Arrêt de Paris du 8 Juin 1387, pour la Duchesse d'Orléans, fille du Roi Charles le Bel ; &amp; est allégué en plaidant le 18 Mars 1388, pour le Comte d'Alençon, que les Princes du sang en font exempts jusqu'au sixième degré."
    content += "Les Pairs de France &amp;"

    return content


def get_TEI_header_footer():
    header = """<?xml version="1.0" encoding="UTF-8"?> \
    \n<TEI xmlns="http://www.tei-c.org/ns/1.0"> \
        \n<teiHeader>\
            \n<fileDesc> \
                \n<titleStmt> \
                    \n<title>Trevoux 1743</title> \
                \n</titleStmt> \
            \n</fileDesc> \
        \n</teiHeader>\
        \n<text> \
            \n<body>\n"""
    
    footer = """\n</body> \
        \n</text> \
    \n</TEI>"""
    return header, footer


def correct_xml_syntax(content):

    # turn <article ID=250000020> into <article ID="250000020">
    content = re.sub(r'<article ID=([0-9]+)>', r'<article ID="\1">', content)

    # tome 1
    # turn <G><S>Anba,</G></S> into <G><S>Anba</S></G> 
    content = content.replace('</G></S>', '</S></G>')
    content = content.replace('<Etym></I>', '</Etym></I>')
    content = content.replace('<REM>', '<REM />') # symbole main
    content = content.replace("<vedette><G>", "<G><vedette>").replace("</G></vedette>", "</vedette></G>")
    # <vedette><G>ABCEDER, (S</G></vedette>')</G>
    content = content.replace("(S</vedette></G>')</G>", "</vedette></G>(S')")
    content = content.replace("<debtab>", "<tab>").replace("<fintab>", "</tab>")
    content = content.replace("<I><cit>","<cit><I>")
    # replace patterns like <symbole= de la racine> or <symbole= deux points en haut et un point en bas> with <symbole />
    content = re.sub(r'<symbole=([^>]+)>', r'<symbole />', content)
    content = content.replace("</cit> <S><auteur>Sanlec</auteur>.</I>", "</I></cit> <S><auteur>Sanlec</auteur>.")
    content = content.replace("<S><auteur>Boil<I></auteur></S>.</I>","<S><auteur>Boil</auteur></S>.")
    # replace patterns like </vedette></G>'ANGLOIS,</G> or </vedette></G> &amp; ANORMAL.</G> with 'ANGLOIS,</vedette></G> or &amp; ANORMAL.</G>
    content = re.sub(r'</vedette></G>([^<]+)</G>', r'\1</vedette></G>', content)
    content = content.replace("<auteur>Hotman.<auteur>", "<auteur>Hotman.</auteur>").replace("<auteur>Liger,<auteur>", "<auteur>Liger,</auteur>")
    content = content.replace("</I></au<I>teur>", "</auteur>")
    content = content.replace("<auteur><I>Matth</auteur></S>. III. I.</I>", "<auteur>Matth</auteur></S><I>. III. I.</I>")
    content = content.replace("<auteur>S. <I>Bernard</auteur></S>,</I>", "<auteur>S. Bernard</auteur></S><I>,</I>")
    content = content.replace("<auteur><I>Juven</auteur></S>.</I>", "<auteur>Juven</auteur></S><I>.</I>")
    content = content.replace("<auteur><I>Ménage</auteur></I>", "<I><auteur>Ménage</auteur></I>")
    content = content.replace("<cit<<I>", "<cit><I>").replace("</cit<", "</cit>")
    content = content.replace("<S><auteur>M. Perrault</auteur>. <S>", "<S><auteur>M. Perrault</auteur>.</S> <S>")
    content = content.replace("<G>ANGÉLIQUES, </G>", "<G><vedette>ANGÉLIQUES, </vedette></G>")
    content = content.replace("<G>ANHIMA, </G>", "<G><vedette>ANHIMA, </vedette></G>")
    content = content.replace("<G>APOS. </G>", "<G><vedette>APOS. </vedette></G>")
    content = content.replace("<G>CANINANA, </G>", "<G><vedette>CANINANA, </vedette></G>")
    



    # tome 2
    content = content.replace("<S><auteur>M.</I> de Claville.</auteur></S>", "<S><auteur>M. de Claville.</auteur></S></I>")
    content = content.replace("<S><auteur>Le</S>", "<S><auteur>Le")
    content = content.replace("{162}<S><svedet>Maître.</auteur></svedet></S>", "{162}Maître.</auteur></S>")
    content = content.replace("<auteur><I>Matth</auteur></S>.</I>", "<auteur>Matth</auteur></S>.") 
    content = content.replace("<S></cit>Id.</S>", "</cit><S>Id.</S>")
    content = content.replace("<oeuvre<", "<oeuvre>") # faire un pattern ?
    content = content.replace("<auteur><I>Cambden</auteur></S>, Ibernia, p.</I>", "<auteur>Cambden</auteur></S>, <I>Ibernia, p.</I>")
    content = content.replace("</cit></I> <S>Id.</S>", "</I></cit> <S>Id.</S>")

    # tome 3
    content = content.replace("<cit><I>Quant aux poissons grands &amp; petits</I>", "<I>Quant aux poissons grands &amp; petits</I>")
    content = content.replace("<I>Que nourrit Madame Thétis</cit>", "<I>Que nourrit Madame Thétis")
    content = content.replace("<auteur><I>Marm</auteur></S>. L. III. C.</I>", "<auteur>Marm</auteur></S>. <I>L. III. C.</I>")
    content = content.replace("</Etym,", "</Etym>,")
    content = content.replace("<I><I>", "</I><I>")
    content = content.replace("<I>Consentant à s'embrasser...</cit>", "<I>Consentant à s'embrasser...</I></cit><I>")
    content = content.replace("HOMOUSIASTE,</G>", "HOMOUSIASTE,")
    content = content.replace("ignorent.</cit>", "ignorent.").replace("vérités.</cit>", "vérités.").replace("pauvreté.</cit>", "pauvreté.").replace("d'eux.</cit> <S><auteur>Mén.","d'eux. <S><auteur>Mén.")
    content = content.replace("{1310}de la <I>honte.</I></cit>", "{1310}de la <I>honte.</I>").replace("quoi.</cit> <S><auteur>Costar.", "quoi. <S><auteur>Costar.")
    content = content.replace("<I><Etym>injuria</Etym>.", "<I><Etym>injuria</Etym></I>.")
    content = content.replace("<auteur><I>Matth</auteur></S>. II.</I>", "<auteur>Matth</auteur></S>. <I>II.</I>")
    content = content.replace("<auteur><I>Matth</auteur></S>. V.</I>", "<auteur>Matth</auteur></S>. <I>V.</I>")
    content = content.replace("Porto di <G><vedette>GORO,</vedette></G>", "Porto di GORO,")
    content = content.replace("<G>HEL. </G>", "<G><vedette>HEL. </vedette></G>")
    

    # tome 4
    
    #content = content.replace("<I><Etym>electum</Etym>, <I>", "<I><Etym>electum</Etym>,</I> <I>")
    content = content.replace("</uvre>", "</oeuvre>")
    #content = content.replace("<I><Etym>longa</Etym>, <I>", "<I><Etym>longa</Etym></I>, <I>")
    content = re.sub(r"(<I><Etym>.*?</Etym>), <I>", r"\1</I>, <I>", content)
    #content = content.replace("</I> est dérivé de <Etym><grec></grec></Etym></I>", "</I> est dérivé de <Etym><grec></grec></Etym>")
    content = content.replace("<I><Etym>manopera</Etym><I>", "<I><Etym>manopera</Etym></I>")
    content = content.replace("<auteur>M. Sc.</Auteur>", "<auteur>M. Sc.</auteur>")
    content = content.replace("<I></Etym>missaticus</Etym></I>", "<I><Etym>missaticus</Etym></I>")
    #content = content.replace("</I> Borel le dérive du Grec <Etym><grec></grec></Etym></I>", "</I> Borel le dérive du Grec <Etym><grec></grec></Etym></I>")
    content = re.sub(r"(</I>.*?<Etym><grec></grec></Etym>)</I>", r"\1", content)
    content = content.replace("depuis le cap d_ GalP´jusqu'à celui de Bongerbino", "depuis le cap de Galle, jusqu'à celui de Bongerbino")
    content = content.replace("<I>oeuvre>", "<I><oeuvre>")
    content = content.replace("</G> le lac <G><vedette>NIPIS,</vedette></G>", "</G> le lac NIPIS,")
    content = content.replace("<G>LOUTRÉE. </G>", "<G><vedette>LOUTRÉE. </vedette></G>")
    content = content.replace("<G>MONS, </G>", "<G><vedette>MONS, </vedette></G>")
    content = content.replace("<G>PANGLOSSIE. </G>", "<G><vedette>PANGLOSSIE. </vedette></G>")


    # tome 5
    content = content.replace("</I> au mot Piailler,</I>", "</I> au mot Piailler,")
    content = content.replace("</I> art. Léonard</I>", "</I> art. Léonard")
    content = content.replace("<I>Satyre contre les perruques.</cit> p.</I>", "<I>Satyre contre les perruques.</I></cit> p.")
    content = content.replace("</oeuvre<</I>", "</oeuvre></I>")
    content = content.replace("<S>de</S> </S>", "<S>de</S> <S>")
    content = content.replace("<c/it>", "</cit>")
    content = content.replace("<I>Maître Gervais....</cit>", "Maître Gervais....</cit>").replace("<oeuvre>Poësies</oeuvre> du</I>", "<oeuvre>Poësies</oeuvre> du")
    content = content.replace("<auteur>Regn<auteur>", "<auteur>Regn</auteur>")
    content = content.replace("<oeuvre>Contes<oeuvre>", "<oeuvre>Contes</oeuvre>")
    content = content.replace("</I><auteur>la Font.</auteur></I>", "<I><auteur>la Font.</auteur></I>")
    content = content.replace("</oeuvre<</svedet>", "</oeuvre></svedet>")

    # tome 6
    #content = content.replace("<auteur><I>Valois</auteur></S>, Not. Gall. p.</I>", "<auteur>Valois</auteur></S>, <I>Not. Gall. p.</I>")
    #content = content.replace("<auteur><I>Fur</auteur></S>, latro. Vargus, Varingus.</I>", "<auteur>Fur</auteur></S>, <I>latro. Vargus, Varingus.</I>")
    content = re.sub(r"<auteur><I>(.*?)</auteur></S>(.*?)</I>", r"<auteur>\1</auteur></S><I>\2</I>", content)
    content = content.replace("</oeuvre<</S>", "</oeuvre></S>")
     
    return content



# div : colonne / page ?


In [3]:
inputpath = os.path.join('..','data','1743_LeRobert', 'input')
outputpath = os.path.join('..', 'data', '1743_LeRobert', 'xml')

for i in range(1, 7):

    files = sorted(os.listdir(inputpath))
    content = ""
    for filename in files:
        if filename.startswith(f'TR{i}-'):
            with open(os.path.join(inputpath,filename), 'r', encoding='latin-1') as f:
                content += f.read()
    if i == 1:
        # supprimer le titre (avant <article)
        pos = content.find('<article')
        content = content[pos:]
    if i == 4:
        # manque fin de la balise article à la fin du TR4
        content += "</article>"
    if i == 5:
        # problème manque 1ère page du TR5.txt
        content = get_1st_page_T5() + content

    header, footer = get_TEI_header_footer()
    content = header + content + footer
    
    # corriger pour produire du XML valide
    content = correct_xml_syntax(content)


    with open(f'{outputpath}/TR{i}.xml', 'w', encoding='utf-8') as f:
        f.write(content)


## Check if the XML are valid

In [4]:
inputpath = os.path.join('..', 'data', '1743_LeRobert', 'xml')

for filename in sorted(os.listdir(inputpath)):
    try:
        parser = etree.XMLParser(collect_ids=False, encoding='utf-8') 
        root = etree.parse(os.path.join(inputpath, filename), parser=parser).getroot()    
        #print(root.nsmap)
        print(filename, ": valide")
        
    except etree.XMLSyntaxError as e:
        print(f"Erreur de syntaxe XML : {e}")

TR1.xml : valide
TR2.xml : valide
TR3.xml : valide
TR4.xml : valide
TR5.xml : valide
TR6.xml : valide
